<a href="https://colab.research.google.com/github/ArashDehghanyan/ml-practicres/blob/main/tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Keras Tuner**

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [10]:
! pip install -q -U keras-tuner

In [11]:
import keras_tuner as kt 

##Download MNIST Dataset

In [12]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

# Normalize pixel values between 0 and 1
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

##Define Model

In [13]:
def model_builder(hp):
    model = keras.models.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    # choose an optimal value betwqeen 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(hp_units, activation='relu'))
    model.add(layers.Dense(10))

    # Tune the learning_rate for optimizer
    # Choose the values 0.01, 0.001 or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model
    

##Instantiate tuner

In [14]:
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='intro_to_kit'
)

In [15]:
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

##**Run Hyperparameter Search**

In [17]:
tuner.search(train_images, train_labels, epochs=50, validation_split=0.2, callbacks=[stop_early])


Trial 31 Complete [00h 01m 10s]
val_accuracy: 0.9787499904632568

Best val_accuracy So Far: 0.9801666736602783
Total elapsed time: 00h 14m 06s
INFO:tensorflow:Oracle triggered exit


In [20]:
# Get the optimal hyper parameter
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

In [21]:
print(f"""Optimal number of units for the first Dense layer: {best_hyperparameters.get('units')}
      Optimal learning rate for the optimizer: {best_hyperparameters.get('learning_rate')}""")

Optimal number of units for the first Dense layer: 512
      Optimal learning rate for the optimizer: 0.001


##**Train Model**

In [22]:
# Build the model with the optimal hyperparameters and train it on data for 50 epochs
model = tuner.hypermodel.build(best_hyperparameters)

# Train model using optimal parameters
history = model.fit(train_images, train_labels, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1

print()
print("Best epoch:", best_epoch)

Epoch 1/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2243 - accuracy: 0.9336 - val_loss: 0.1232 - val_accuracy: 0.9630
Epoch 2/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0868 - accuracy: 0.9732 - val_loss: 0.0928 - val_accuracy: 0.9718
Epoch 3/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0565 - accuracy: 0.9822 - val_loss: 0.0799 - val_accuracy: 0.9773
Epoch 4/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0400 - accuracy: 0.9872 - val_loss: 0.0781 - val_accuracy: 0.9773
Epoch 5/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0282 - accuracy: 0.9912 - val_loss: 0.0988 - val_accuracy: 0.9720
Epoch 6/50
1500/1500 [==============================] - 8s 6ms/step - loss: 0.0225 - accuracy: 0.9924 - val_loss: 0.0892 - val_accuracy: 0.9773
Epoch 7/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.0176 - accuracy: 0.9944 - val_loss: 0.0908 - val_accuracy:

In [23]:
# Reinstantiate the hypermodel and train it with the optimal number of epochs
hypermodel = tuner.hypermodel.build(best_hyperparameters)

# Re-train the model
hypermodel.fit(train_images, train_labels, epochs=best_epoch, validation_split=0.2)

Epoch 1/18
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2222 - accuracy: 0.9338 - val_loss: 0.1136 - val_accuracy: 0.9645
Epoch 2/18
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0870 - accuracy: 0.9731 - val_loss: 0.1047 - val_accuracy: 0.9690
Epoch 3/18
1500/1500 [==============================] - 8s 6ms/step - loss: 0.0569 - accuracy: 0.9831 - val_loss: 0.0954 - val_accuracy: 0.9715
Epoch 4/18
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0386 - accuracy: 0.9880 - val_loss: 0.0729 - val_accuracy: 0.9776
Epoch 5/18
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0283 - accuracy: 0.9908 - val_loss: 0.0972 - val_accuracy: 0.9730
Epoch 6/18
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0231 - accuracy: 0.9925 - val_loss: 0.0874 - val_accuracy: 0.9772
Epoch 7/18
1500/1500 [==============================] - 8s 6ms/step - loss: 0.0162 - accuracy: 0.9946 - val_loss: 0.0845 - val_accuracy:

In [24]:
# Evaluate model on the test set
evaluate_result = hypermodel.evaluate(test_images, test_labels)
print("[test loss, test accuracy]:", evaluate_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.1157 - accuracy: 0.9787
[test loss, test accuracy]: [0.11569742113351822, 0.9786999821662903]
